In [ ]:
from odc.aws.queue import publish_messages, get_queue
from datetime import datetime, timedelta
import os
import json

In [ ]:
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"
dead_queue = get_queue("ghrsst-queue-dead")
queue = get_queue("ghrsst-queue")

In [ ]:
import tqdm

# For each day of the year, create a timestamp string and publish it to the queue
messages = []
for year in [2025]:
    for day in range(1, 45):
        dt = datetime(year, 1, 1) + timedelta(day - 1)
        body = {
            "date": dt.strftime("%Y-%m-%d"),
        }
        messages.append({"Id": body["date"], "MessageBody": json.dumps(body)})

count = 0
message_group = []
progress = tqdm.tqdm(messages)

for message in progress:
    # Assume this works. Exception handling elsewhere.
    message_group.append(message)
    count += 1

    if count % 10 == 0:
        publish_messages(queue, message_group)
        message_group = []

# Post the last few messages
if len(message_group) > 0:
    publish_messages(queue, message_group)
    progress.update(count)

# Return the number of messages that were re-driven.
print(f"Published {count} messages")

In [ ]:
# dead_queue = get_queue("ghrsst-queue-dead")
# queue = get_queue("ghrsst-queue")

print(
    f"Found {dead_queue.attributes.get('ApproximateNumberOfMessages')} messages in dead queue"
)
print(
    f"Found {queue.attributes.get('ApproximateNumberOfMessages')} messages in the alive queue"
)

In [ ]:
import os
import boto3
from datetime import datetime, timedelta
import json

N_PREVIOUS_DAYS = 10

QUEUE_NAME = "ghrsst-queue"
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"


today = datetime.today()

messages = []

for i in range(N_PREVIOUS_DAYS):
    dt = today - timedelta(days=i)

    body = {
        "date": dt.strftime("%Y-%m-%d"),
    }
    messages.append({"Id": body["date"], "MessageBody": json.dumps(body)})

sqs = boto3.resource("sqs")
queue = sqs.get_queue_by_name(QueueName=QUEUE_NAME)

queue.send_messages(Entries=messages)